In [1]:
from IPython.display import clear_output
from google.colab import files
files.upload()
clear_output()

In [2]:
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 66 Jun 17 09:02 kaggle.json
/content


In [3]:
##### DOWNLOAD DATASET #####
!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

In [3]:
import pandas as pd

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext

In [30]:
#TODO do it using SPARK directly
actors = pd.read_csv("letterbox/actors.csv")
baskets = actors.groupby("id")["name"].apply(list)

In [31]:
print("number of baskets: " + str(len(basket)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)

number of baskets: 603163
biggest basket: 451
id
1000001    [Margot Robbie, Lewis Easter, Onyemachi Ejimof...
1000002    [Rosie Peralta, Anna Elisabeth Rihlmann, Andre...
1000003    [Randall Archer, Boon Pin Koh, Efka Kvaracieju...
1000004    [Leonard Termo, Greg Bronson, Michael Arturo, ...
1000005    [Lena Georgas, Jeff Hephner, Elyes Gabel, Broo...
                                 ...                        
1896377    [Marat Basharov, Sergey Rost, Gleb Kalyuzhny, ...
1896382                                   [Marine Petrosyan]
1896387                                        [Rebecca Jim]
1896391                                       [Ko Shibasaki]
1896393                                    [Atsuhiro Inukai]
Name: name, Length: 603163, dtype: object


In [ ]:
## APRIORI
def apriori():